In [1]:
import requests
from bs4 import BeautifulSoup as bs
from collections import deque
import time
import pickle
import random
from multiprocessing import Pool
#import requesocks
from collections import Counter

In [131]:
def scrape(isbn):
    time.sleep(random.random())
    #session = requesocks.session()
    #session.proxies = {
    #'http': 'socks5://127.0.0.1:9050',
    #'https': 'socks5://127.0.0.1:9050'}
    url = 'http://www.barnesandnoble.com/w/1100101506?ean={}'.format(isbn)
    #html = session.get(url)
    html = requests.get(url)
    soup = bs(html.text)
    ids=url.split('/')[-1].split('?')[0]
    try:
        isbns=soup.find("dt",text="ISBN-13:").findNextSibling("dd").getText()
    except:
        isbns=''
    try:
        titles=soup.findAll('h1',{'itemprop':"name"})[0].getText()
    except:
        titles=''
    try:
        authors=soup.find('span',{'class':"contributors"}).find('a').getText()
    except:
        authors=''
    try:
        dates=soup.find("dt",text="Publication date:").findNextSibling("dd").getText()
    except:
        dates=''
    try:
        editions=soup.find("dt",text="Edition description:").findNextSibling("dd").getText()
    except:
        editions=''
    try:
        pages=soup.find("dt",text="Pages:").findNextSibling("dd").getText()
    except:
        pages=''
    try:
        saleRanks=soup.find("dt",text="Sales rank:").findNextSibling("dd").getText()
    except:
        saleRanks=''
    edges=[]
    try:
        recommendationURL='http://www.barnesandnoble.com/includes/bn-recommendation-content.jsp?count=100&fromWL=&title=Customers+Who+Bought+This+Item+Also+Bought&skuId={}&_={}'.format(isbn,ids)
        html = requests.get(recommendationURL)
        soup = bs(html.text)
        for i in soup.find_all('a',{'class':"book-carousel-link"}):
            u=i.get('href')
            edges.append((isbn,u.split('=')[-1]))
    except:
        pass
    return (html.status_code,(ids,isbns,titles,authors,dates,editions,pages,saleRanks,edges))

In [ ]:
isbns=pickle.load(open('../BookCodes/isbn2isbn13.pkl'))
querySet = list(set(zip(*isbns)[1]))

pool=Pool(30)
results=pool.map(scrape,querySet,len(querySet)/30)
pool.close()
pool.join()

code,data=zip(*results)
books=dict(zip(querySet,data))
statusCode=dict(zip(querySet,code))

with open('../BarnesNobel/scienceBooks.pkl','wb') as outfile:
    pickle.dump(books,outfile)

In [450]:
querySet=[i for i in statusCode if statusCode[i]!=200]

In [441]:
with open('../BarnesNobel/scienceBooks.txt','w') as outfile:
    for i in querySet:
        outfile.write(i+'\n')

* The following part is replaced by the spark-ec2 scrapper

In [ ]:
pool=Pool(30)
results=pool.map(scrape,querySet,len(querySet)/30)
pool.close()
pool.join()

code,data=zip(*results)
books=dict(zip(querySet,data))
statusCode=dict(zip(querySet,code))

with open('../BarnesNobel/scienceBooks_ext1.pkl','wb') as outfile:
    pickle.dump(dict(zip(querySet,data)),outfile)

with open('../BarnesNobel/scienceBooks_resp.pkl','wb') as outfile:
    pickle.dump(dict(zip(querySet,code)),outfile)

* Get results from ec2 and put them back in 

In [443]:
newdata=pickle.load(open('../BarnesNobel/scienceBooks_ext15.pkl'))

In [445]:
for i in newdata:
    books[i]=newdata[i]

In [446]:
resp = pickle.load(open('../BarnesNobel/scienceBooks_resp.pkl'))

In [447]:
for i in resp:
    statusCode[i]=resp[i]

In [448]:
with open('../BarnesNobel/scienceBooks.pkl','wb') as outfile:
    pickle.dump(books,outfile)

* Build graph

In [9]:
from igraph import *

In [4]:
isbn2asin = pickle.load(open('../AmazonBooks/isbn_lookup.pkl'))
books = pickle.load(open('../BarnesNobel/scienceBooks.pkl'))
isbns=pickle.load(open('../BookCodes/isbn2isbn13.pkl'))

In [5]:
emptyBooks=[i for i in books if not books[i][1]]
isolates=[i for i in books if not books[i][-1]]
len(emptyBooks), len(isolates), len(books), len(isbns)

(22469, 172982, 469668, 469669)

In [6]:
isbn10,isbn13=zip(*isbns)
isbn13to10=dict(zip(isbn13,isbn10))

In [7]:
def isbnConverter(isbn13):
    isbn10=isbn13[3:-1]
    m=10
    check=0
    for i in isbn10:
        check+=int(i)*m
        m-=1
    last=(11-(check % 11)) % 11
    if last==10:
        isbn10=isbn13[3:-1]+'X'
        if isbn10 not in isbn2asin:
            isbn10=isbn13[3:-1]+'x'
    else:
        isbn10=isbn13[3:-1]+str(last)
    return isbn10

In [10]:
edgeList=[(isbn2asin[isbnConverter(e[0])],isbn2asin[isbnConverter(e[1])]) for i in books for e in books[i][-1] if (isbnConverter(e[0]) in isbn2asin and isbnConverter(e[1]) in isbn2asin)]
nodes=set([i for e in edgeList for i in e])
g = Graph()
g.add_vertices(list(nodes))
g.add_edges(edgeList)
g.simplify()
g.summary()

'IGRAPH UN-- 439603 3375406 -- \n+ attr: name (v)'

In [11]:
asin2id=dict()
for u in g.vs:
    asin2id[u['name']]=u.index

In [ ]:
pickle.dump(g,open('../BarnesNobel/asin_network.pkl','wb'))

In [ ]:
outfile=open('../BarnesNobel/copurchase_list.txt','w')
for e in g.es:
    outfile.write('{} {}\n'.format(g.vs[e.source]['name'],g.vs[e.target]['name']))
outfile.close()

In [12]:
pickle.dump(asin2id,open('../BarnesNobel/asin2id.pkl','wb'))